In [1]:
from torch import nn
import torch
from torch.utils.data import DataLoader
import utils

In [2]:
#@save
class Seq2Seq(nn.Module):
    def __init__(self, enc_v_dim, dec_v_dim, emb_dim, units, max_pred_len, start_token, end_token):
        # 输入参数（编码器词表大小，解码器词表大小，嵌入向量维度，隐藏层数量，最大预测长度，开始标记，结束标记）
        super(Seq2Seq, self).__init__()
        self.units = units
        self.dec_v_dim = dec_v_dim

        #encoder
        self.enc_embeddings = nn.Embedding(enc_v_dim, emb_dim)
        #enc_v_dim：表示输入序列（例如文本数据）的词汇表大小，即需要进行嵌入的不同词汇的数量。这个参数必须是一个整数。
        # emb_dim：表示嵌入向量的维度，即每个词汇被嵌入到的向量空间的维度大小。这个参数必须是一个整数。
        self.enc_embeddings.weight.data.normal_(0, 0.1)
        self.encoder = nn.LSTM(emb_dim, units, 1, batch_first=True)
        #在每个时间步骤中接收一个嵌入向量，然后将其传递给 LSTM 层进行处理，最终生成一个隐藏状态张量。整个过程将在训练期间自动完成。
        # emb_dim: 输入嵌入向量的维度。
        # units: LSTM 层中隐藏单元的数量（也称为 LSTM 的单元数或输出维度）。
        # 1: LSTM 层的数量。此处只有一个 LSTM 层。
        # batch_first=True: 这表示在输入张量中批次（batch）的维度应该是第一维。因此，输入张量的形状应该是 (batch_size, seq_length, emb_dim)。

        #decoder
        self.dec_embeddings = nn.Embedding(dec_v_dim, emb_dim)
        self.dec_embeddings.weight.data.normal_(0, 0.1)
        self.decoder_cell = nn.LSTMCell(emb_dim, units)
        #这段代码实例化了一个LSTM单元（LSTMCell）对象。输入参数emb_dim指定了输入的特征向量维度，而units则指定了LSTM单元中隐藏状态的维度。
        # 与此不同，nn.LSTM(emb_dim, units, 1, batch_first=True)创建了一个完整的LSTM层对象，其中emb_dim指定了输入序列中每个时间步的特征向量维度，units指定了LSTM层中隐藏状态的维度，1代表该LSTM层只有一层，batch_first=True则表示输入数据的第一维为批次大小。
        # LSTMCell是LSTM层的基本组成部分，它只处理输入序列中的一个时间步，因此在使用LSTMCell时需要手动将序列中的每个时间步输入到LSTMCell中。而nn.LSTM则可以自动处理序列中的所有时间步，因此它更适合处理可变长度的序列数据。
        self.decoder_dense = nn.Linear(units, dec_v_dim)

        self.opt = torch.optim.Adam(self.parameters(), lr=0.001)
        self.max_pred_len = max_pred_len
        self.start_token = start_token
        self.end_token = end_token

        # 这段代码是一个方法，接收一个输入张量x，并返回编码器网络处理后的隐藏状态。
        # - `embedded = self.enc_embeddings(x)`: 使用编码器嵌入层将输入张量x转换为嵌入向量，其中`self.enc_embeddings`是一个嵌入层对象，其调用可以将x中的整数索引映射到对应的向量表示。结果嵌入向量embedded的形状为 [n,step,emb]，其中n为batch size，step为序列长度，emb为嵌入向量维度。
        # - `hidden = (torch.zeros(1,x.shape[0],self.units),torch.zeros(1,x.shape[0],self.units))`: 创建初始的隐藏状态，即LSTM的前一时刻的输出和记忆单元状态。这里使用了torch.zeros()来创建全零张量，第一个维度为1表示只有一个LSTM层，第二个维度为x的batch size，第三个维度为self.units即LSTM的隐藏单元个数。
        # - `o,(h,c)=self.encoder(embedded,hidden)`: 将嵌入向量和初始隐藏状态传递给编码器网络，得到最终的隐藏状态和输出。self.encoder是LSTM编码器模型对象，其调用可以将嵌入向量序列和隐藏状态序列作为输入，返回最终的输出序列和隐藏状态。这里的输出只取了最后一个时间步的隐藏状态和记忆单元状态，即h和c。
        # - `return h,c`: 返回最终的隐藏状态h和记忆单元状态c。

    #对输入序列进行编码，返回最后一个时间步的隐藏状态 h 和细胞状态 c，这些状态包含了编码器对输入序列的理解和提取的信息。
    def encode(self,x):
        embedded = self.enc_embeddings(x)   # [n,step,emb]，n：批次大小，step：输入序列的长度，emb：嵌入维度
        hidden = (torch.zeros(1,x.shape[0],self.units),torch.zeros(1,x.shape[0],self.units))
        o,(h,c)=self.encoder(embedded,hidden)
        return h,c  #h/c的维度：(num_layers, batch_size, hidden_size),[LSTM层的数量,输入样本的批量大小,隐藏状态的维度]

    #用于模型的推断过程，即在训练完成后使用模型进行预测
    #生成模型在给定输入数据上的预测输出，用于模型的评估或应用
    def inference(self,x):
        self.eval() #将模型设为评估模式，即关闭 Dropout 和 Batch Normalization 的训练行为
        hx,cx=self.encode(x)   #对输入序列x进行编码
        hx,cx=hx[0],cx[0]       #获取编码器的最终隐藏状态
        start = torch.ones(x.shape[0],1)    #创建一个形状为 (batch_size, 1) 的张量 start，用于表示解码器的起始输入
        start[:,0]=torch.tensor(self.start_token)   #将其中的值都设为 self.start_token
        start = start.type(torch.LongTensor)
        dec_emb_in=self.dec_embeddings(start)   #对起始输入 start 进行解码器的嵌入层的转换，得到解码器的输入嵌入向量
        dec_emb_in=dec_emb_in.permute(1,0,2)    #将时间步维度移动到第一个维度
        dec_in=dec_emb_in[0]    #提取出第一个时间步的输入向量 dec_in
        output=[]
        for i in range(self.max_pred_len):
            hx, cx = self.decoder_cell(model.inference(bx[0:1]), (hx, cx))    #将解码器的输入 dec_in 和前一个时间步的隐藏状态 (hx, cx) 传递给解码器的 LSTMCell 层，得到新的隐藏状态 (hx, cx)
            o = self.decoder_dense(hx)  #将隐藏状态映射为输出向量 o
            o = o.argmax(dim=1).view(-1,1)  #取得每个样本在当前时间步的预测结果，并将其视为下一个时间步的输入
            dec_in=self.dec_embeddings(o).permute(1,0,2)[0]
            output.append(o)
        output = torch.stack(output,dim=0)  #将所有时间步的输出结果存储在张量 output 中
        self.train()    #将模型设为训练模式

        return output.permute(1,0,2).view(-1,self.max_pred_len) #(batch_size, max_pred_len)
        # permutte(1,0,2)：这是一个张量的维度重排操作，它将输入张量的第二维和第一维交换位置，不改变其他维度的顺序。例如，如果输入张量是 shape 为 [batch_size, seq_len, hidden_size] 的三维张量，则交换第一维和第二维后，输出张量的 shape 将变为 [seq_len, batch_size, hidden_size]。
        # view(-1, self.max_pred_len)：这是一个张量的形状调整操作，它将输入张量的形状调整为 (未知长度, self.max_pred_len) 的二维形状，其中 -1 表示剩余所有元素的数量。例如，如果输入张量的形状是 [seq_len, batch_size, hidden_size]，则通过 view(-1, self.max_pred_len) 操作，可以将其重新调整为二维张量，其中每一行代表一个序列（从原来的不同位置提取）并且每个序列限制为最大预测长度。

    #在训练过程中使用，通常用于计算模型在训练数据上的损失和梯度
    #生成模型的预测结果，通常用于训练过程中的前向传播或评估阶段
    def train_logit(self,x,y):
        hx,cx=self.encode(x)    #对输入序列x进行编码
        hx,cx=hx[0],cx[0]   #获取编码器的最终隐藏状态
        dec_in = y[:,:-1]   #从目标序列y中移除最后一个时间步，得到解码器的输入序列dec_in
        dec_emb_in = self.dec_embeddings(dec_in)
        dec_emb_in=dec_emb_in.permute(1,0,2)   # 调整维度顺序为(step, n, emb)以适应解码器的输入要求
        output=[]
        for i in range(dec_emb_in.shape[0]):#遍历解码器嵌入层输入的每个时间步
            hx,cx=self.decoder_cell(dec_emb_in[i],(hx,cx))  #lstmcell
            o=self.decoder_dense(hx)     #linear
            output.append(o)
        output = torch.stack(output,dim=0)#将输出列表中的张量按指定维度堆叠起来，得到形状为 (step, n, dec_v_dim) 的输出张量
        return output.permute(1,0,2)    # (n, step, dec_v_dim)

    def step(self,x,y):
        self.opt.zero_grad()
        batch_size = x.shape[0]
        logit = self.train_logit(x,y)#前向传播
        dec_out = y[:,1:]
        loss = model.step(bx,by)#计算交叉熵损失（损失函数计算）
        loss.backward()#反向传播
        self.opt.step()#根据计算得到的梯度更新模型的参数（更新参数）
        return loss.detach().numpy()

In [3]:
# 这段代码的目的是进行模型的训练和评估。每个批次都会执行一次训练过程，并记录训练损失。此外，每隔一定批次，会打印出一个样例的输入、目标和预测结果，用于观察模型的训练效果。循环执行100个轮次，完成整个训练过程。
dataset = utils.DateData(4000)
print("Chinese time order: yy/mm/dd ",dataset.date_cn[:3],"\nEnglish time order: dd/M/yyyy", dataset.date_en[:3])
print("Vocabularies: ", dataset.vocab)
print(f"x index sample:  \n{dataset.idx2str(dataset.x[0])}\n{dataset.x[0]}",
f"\ny index sample:  \n{dataset.idx2str(dataset.y[0])}\n{dataset.y[0]}")

Chinese time order: yy/mm/dd  ['31-04-26', '04-07-18', '33-06-06'] 
English time order: dd/M/yyyy ['26/Apr/2031', '18/Jul/2004', '06/Jun/2033']
Vocabularies:  {'Oct', '2', '9', 'Nov', '3', 'Mar', '4', '<EOS>', '6', 'Dec', '<GO>', '8', 'Feb', '7', '/', 'May', '0', '<PAD>', 'Aug', '1', '5', 'Jun', 'Jan', 'Sep', '-', 'Jul', 'Apr'}
x index sample:  
31-04-26
[6 4 1 3 7 1 5 9] 
y index sample:  
<GO>26/Apr/2031<EOS>
[14  5  9  2 15  2  5  3  6  4 13]


In [9]:
print(dataset.x)
print(dataset.y)
print(dataset.i2v)
print(dataset.v2i)

[[ 6  4  1 ...  1  5  9]
 [ 3  7  1 ...  1  4 11]
 [ 6  6  1 ...  1  3  9]
 ...
 [ 5 11  1 ...  1  3  6]
 [10 10  1 ...  1  3  7]
 [ 5  3  1 ...  1  3  7]]
[[14  5  9 ...  6  4 13]
 [14  4 11 ...  3  7 13]
 [14  3  9 ...  6  6 13]
 ...
 [14  3  6 ...  5 11 13]
 [14  3  7 ... 10 10 13]
 [14  3  7 ...  5  3 13]]
{1: '-', 2: '/', 3: '0', 4: '1', 5: '2', 6: '3', 7: '4', 8: '5', 9: '6', 10: '7', 11: '8', 12: '9', 13: '<EOS>', 14: '<GO>', 15: 'Apr', 16: 'Aug', 17: 'Dec', 18: 'Feb', 19: 'Jan', 20: 'Jul', 21: 'Jun', 22: 'Mar', 23: 'May', 24: 'Nov', 25: 'Oct', 26: 'Sep', 0: '<PAD>'}
{'-': 1, '/': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, '<EOS>': 13, '<GO>': 14, 'Apr': 15, 'Aug': 16, 'Dec': 17, 'Feb': 18, 'Jan': 19, 'Jul': 20, 'Jun': 21, 'Mar': 22, 'May': 23, 'Nov': 24, 'Oct': 25, 'Sep': 26, '<PAD>': 0}


In [16]:
print(dataset.start_token)
print(dataset.end_token)
print(dataset.idx2str((21, 12, 14,)))

14
13
Jun9<GO>


In [19]:
loader = DataLoader(dataset,batch_size=32,shuffle=True) #创建一个加载器用于加载数据，将数据打乱后，每32为一批次进行加载
print(loader.dataset==dataset)  #这里的dataset跟torch的函数加载出来的是同一个，即loader其实就是dataset数据集的封装

True


In [ ]:
# 创建模型
model = Seq2Seq(dataset.num_word,dataset.num_word,emb_dim=16,units=32,max_pred_len=11,start_token=dataset.start_token,end_token=dataset.end_token)
# 对模型进行训练和评估
for i in range(100):    #进行100个训练轮次
    for batch_idx , batch in enumerate(loader): #遍历数据加载器中的每个批次
        bx, by, decoder_len = batch #从当前批次中解包出输入数据bx，目标数据by和解码器长度decoder_len
        bx = bx.type(torch.LongTensor)  #将数据格式转换为LongTensor类型
        by = by.type(torch.LongTensor)
        loss = model.step(bx,by)    #执行一步训练过程，并返回训练损失
        # 每个一定批次，执行依次输出
        if batch_idx % 70 == 0: #将目标序列转化为文本形式
            target = dataset.idx2str(by[0, 1:-1].data.numpy())
            pred = model.inference(bx[0:1]) #使用模型的inference方法对输入数据进行预测
            res = dataset.idx2str(pred[0].data.numpy())#将预测结果转换为文本形式（以便输出）
            src = dataset.idx2str(bx[0].data.numpy())#将输入数据转换为文本形式（以便输出）
            print(
                "Epoch: ",i,
                "| t: ", batch_idx,
                "| loss: %.3f" % loss,
                "| input: ", src,
                "| target: ", target,
                "| inference: ", res,
            )